In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [2]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
df.drop(columns=['day'])
# display the table


,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
29/02/20,105
01/03/20,140
02/03/20,166
03/03/20,229
04/03/20,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [3]:
fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases')
fig.show()

[0.65       0.55923994 1.        ]
`ftol` termination condition is satisfied.
Function evaluations 14, initial cost 1.5458e+01, final cost 3.1784e-02, first-order optimality 2.24e-07.
            Exponential model
            x0 = -350.81595468302237
            y0 = -4559.223467748227
            k = 0.020550345340093763
            
`ftol` termination condition is satisfied.
Function evaluations 18, initial cost 4.4436e+00, final cost 3.5151e-04, first-order optimality 1.44e-08.
        Sigmoid model
        x0 = 77.30899926307048
        y0 = -65.66625013578133
        c = 4425.824537056226
        k = 0.18168941537569644
        asymptot = 4360.158286920445
        flex = 77.30899926307048, 2147.2460183923317
        
`ftol` termination condition is satisfied.
Function evaluations 30, initial cost 1.9795e+00, final cost 1.2824e-03, first-order optimality 5.18e-08.
        Sigmoid derivative model
        x0 = 90.9042090056176
        y0 = -306.1975251181727
        c = 145239.78503